<a href="https://colab.research.google.com/github/jdeiros/soa-2020/blob/master/HPC/Deiros_Jeronimo_ejercicio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Introducción
Por lo general, las computadoras se utilizan para compilar y analizar los resultados de encuestas y estudios de opinión.

El siguiente cuaderno calcula la media y la moda de los N valores de los elementos de un vector (resultados), lo hace en forma secuencial y en tambien en paralelo utilizando la biblioteca omp en el lenguaje C, que corre en la plataforma Colab ejecutado desde codigo python. 

Cada elemento del vector se inicializa con resultados aleatorios de puntajes (numeros enteros entre 0 y 9).

El algoritmo para el calculo de la media y la moda se basa en un ejemplo práctico del libro "C/C++ Cómo Programar"[1]

Su objetivo es aprender a utilizar Python[2] en la plataforma Colab [3] y la programación secuencial y en paralelo con openMP.


#2. Armado de ambiente
No es necesario para este ejercicio.

#3. Desarrollo

## 3.1 Creación en python del archivo que tiene el codigo C para la ejecución del programa encuesta.

In [143]:
# Codigo Python, que tiene el código C de la ejecución del programa encuesta.
code = """
#include <iostream>
#include <vector>
#include <cstdlib>
#include <sys/time.h>
#include <omp.h>

// ----------------------------------------------------------------------------
// Macros que miden el tiempo.

static double dHashTiempoHistory[3];
static struct timeval tv;

#define TIEMPO_INI( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = tv.tv_sec + tv.tv_usec/1000000.0;
   
   
#define TIEMPO_FIN( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = ((tv.tv_sec + tv.tv_usec/1000000.0) - dHashTiempoHistory[ h ]) * 1000; // Devuelvo en milisegundos
#define TIEMPO_GET( h ) dHashTiempoHistory[ h ]

#define HTH_TOTAL         1
#define HTH_MODA_SEC      2
#define HTH_MODA_OMP      3
#define RANGO_PUNTAJES    10
// ----------------------------------------------------------------------------

int main(int argc, char* argv[]) 
{ 
	int i,c;
	TIEMPO_INI( HTH_TOTAL )

	// Leo los parametros.
	if( argc != 2 )
	{
		std::cerr<< " Error en los parametros de indicar: (Tamanio vector resultados)."<<argc<<std::endl;
		exit( -1 );
	}

	int cantidad_elementos = atoi( argv[1] );	
	double sumatoria_secuencial = 0;
	double sumatoria_paralelo = 0;
	// --------------------------------------------
	// Defino la memoria de los vectores.
	std::vector<int> resultados(cantidad_elementos);
	std::vector<int> frecuencias_secuencial( RANGO_PUNTAJES );
	std::vector<int> frecuencias_paralelo( RANGO_PUNTAJES );

	//Inicializo en cero los vectores de frecuencias
	for(int i=0; i < frecuencias_secuencial.size(); i++)
		frecuencias_secuencial[i]=0;
	for(int i=0; i < frecuencias_paralelo.size(); i++)
		frecuencias_paralelo[i]=0;

	//Cargo los puntajes aleatorios (0 a 9) en cada elemento del vector resultados.
	srand ( time(NULL) );
	for (int i=0;i<resultados.size();i++)
		resultados[i] = (int)(rand()%RANGO_PUNTAJES);

	// --------------------------------------------
	// Realizo el calculo de frecuencias y la sumatoria en forma secuencial.

	TIEMPO_INI( HTH_MODA_SEC )
	for(i=0;i < resultados.size(); i++)
	{
		sumatoria_secuencial += resultados[i];
		++frecuencias_secuencial[resultados[i]];
	}
	TIEMPO_FIN( HTH_MODA_SEC )

	// --------------------------------------------
	// Realizo el calculo de frecuencias y la sumatoria con OMP.

	TIEMPO_INI( HTH_MODA_OMP )

	#pragma omp parallel for
	for(i=0;i<resultados.size();i++)
	{
		sumatoria_paralelo += resultados[i];
		++frecuencias_paralelo[resultados[i]];
	}
	TIEMPO_FIN( HTH_MODA_OMP )
	
	
	// Obtengo la moda del vector_frecuencias obtenido secuencialmente
	int mas_grande_secuencial = 0;
	int valor_moda_secuencial = 0;
	for(i=0; i < frecuencias_secuencial.size(); i++)
	{
		if(frecuencias_secuencial[i] > mas_grande_secuencial)
		{
			mas_grande_secuencial += frecuencias_secuencial[i];
			valor_moda_secuencial = i;
		}
	}
	
	// Obtengo la moda del vector_frecuencias obtenido en forma paralela con omp
	int mas_grande_paralelo = 0;
	int valor_moda_paralelo = 0;
	for(i=0; i < frecuencias_paralelo.size(); i++)
	{
		if(frecuencias_paralelo[i] > mas_grande_paralelo)
		{
			mas_grande_paralelo += frecuencias_paralelo[i];
			valor_moda_paralelo = i;
		}
	}
	
	// --------------------------------------------
	// Muestro los resultados.
	std::cout<< "****************************************************************************************" <<std::endl;
	
	std::cout<<"resultados : ["; 
	for(i=0;i<resultados.size();i++)
	{
		std::cout<<resultados[i]<< ", ";
	}
	std::cout<<"]"<<std::endl; 

	std::cout<<"frecuencias_secuencial : [";
	for(i=0;i<frecuencias_secuencial.size();i++)
	{
		std::cout<<frecuencias_secuencial[i]<< ", ";
	}
	std::cout<<"]"<<std::endl; 

	std::cout<<"frecuencias_paralelo : [";
	for(i=0;i<frecuencias_paralelo.size();i++)
	{
		std::cout<<frecuencias_paralelo[i] << ", ";
	}
	std::cout<<"]"<<std::endl; 

	std::cout<<"sumatoria_secuencial: "<<sumatoria_secuencial<<std::endl;
	std::cout<<"media_secuencial: "<<sumatoria_secuencial/cantidad_elementos<<std::endl;
	std::cout<<"sumatoria_paralelo: "<<sumatoria_paralelo<<std::endl; 
	std::cout<<"media_paralelo: "<<sumatoria_paralelo/cantidad_elementos<<std::endl;
	
	std::cout<<"mas_grande_secuencial: "<<mas_grande_secuencial<<std::endl; 
	std::cout<<"valor_moda_secuencial: "<<valor_moda_secuencial<<std::endl;
	std::cout<<"mas_grande_paralelo: "<<mas_grande_paralelo<<std::endl; 
	std::cout<<"valor_moda_paralelo: "<<valor_moda_paralelo<<std::endl;

	TIEMPO_FIN( HTH_TOTAL )
	
	std::cout<<"****************************************************************************************" <<std::endl;
	std::cout<<"Valores Reales  :" <<std::endl;
	std::cout<<"Tiempo TOTAL     : "<<TIEMPO_GET(HTH_TOTAL   )<<" [ms]"<<std::endl;
	std::cout<<"Tiempo encuesta Sec  : "<<TIEMPO_GET(HTH_MODA_SEC)<<" [ms]"<<std::endl;
	std::cout<<"Tiempo encuesta Omp  : "<<TIEMPO_GET(HTH_MODA_OMP)<<" [ms]"<<std::endl;
	std::cout<<std::endl;
	std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_MODA_SEC)<<" / "<<TIEMPO_GET(HTH_MODA_OMP)<<" = "<<TIEMPO_GET(HTH_MODA_SEC)/TIEMPO_GET(HTH_MODA_OMP)<<std::endl;
	std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_MODA_SEC)/TIEMPO_GET(HTH_MODA_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_MODA_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_MODA_OMP))<<std::endl;
	std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_MODA_SEC)<<" = "<<TIEMPO_GET(HTH_MODA_SEC)<<std::endl;
	std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_MODA_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_MODA_OMP)<<std::endl;
	std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_MODA_OMP)<<" - "<<TIEMPO_GET(HTH_MODA_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_MODA_OMP))-TIEMPO_GET(HTH_MODA_SEC)<<std::endl;


	std::cout<<std::endl;
	std::cout<<"Valores Ideal: "<<std::endl;
	TIEMPO_GET(HTH_MODA_OMP) = TIEMPO_GET(HTH_MODA_SEC) / 2;
	std::cout<<"Tiempo encuesta Sec  : "<<TIEMPO_GET(HTH_MODA_SEC)<<" [ms]"<<std::endl;
	std::cout<<"Tiempo encuesta Omp  : "<<TIEMPO_GET(HTH_MODA_OMP)<<" [ms]"<<std::endl;

	std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_MODA_SEC)<<" / "<<TIEMPO_GET(HTH_MODA_OMP)<<" = "<<TIEMPO_GET(HTH_MODA_SEC)/TIEMPO_GET(HTH_MODA_OMP)<<std::endl;
	std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_MODA_SEC)/TIEMPO_GET(HTH_MODA_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_MODA_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_MODA_OMP))<<std::endl;
	std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_MODA_SEC)<<" = "<<TIEMPO_GET(HTH_MODA_SEC)<<std::endl;
	std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_MODA_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_MODA_OMP)<<std::endl;
	std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_MODA_OMP)<<" - "<<TIEMPO_GET(HTH_MODA_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_MODA_OMP))-TIEMPO_GET(HTH_MODA_SEC)<<std::endl;

}
// ----------------------------------------------------------------------------
"""
try:
  text_file = open("code_encuesta.cpp", "w")
  text_file.write(code)
  text_file.close()
except Exception as e:
  print(f"error: {e}")

## 3.2. Compilación de código C del programa encuesta

In [144]:
!g++ -o encuesta -fopenmp code_encuesta.cpp

## 3.3. Ejecución del programa encuesta

In [161]:
# --------------------------------------------
#@title 3.3.1 Parámetros de ejecución { vertical-output: true }

cantidad_elementos =   100#@param {type: "number"}
# --------------------------------------------
try:
  script_file = f"./encuesta {cantidad_elementos}"
  text_file = open("script_file.sh", "w")
  text_file.write(script_file)
  text_file.close()
except Exception as e:
  print(f"error: {e}")
  
%env OMP_NUM_THREADS=2

!chmod 755 ./script_file.sh
!./script_file.sh

env: OMP_NUM_THREADS=2
****************************************************************************************
resultados : [3, 7, 8, 6, 8, 9, 8, 0, 8, 7, 1, 6, 5, 9, 2, 8, 6, 8, 0, 0, 0, 6, 9, 7, 8, 9, 1, 6, 6, 1, 7, 1, 0, 7, 8, 1, 6, 8, 3, 6, 5, 4, 4, 2, 4, 9, 0, 2, 7, 2, 2, 9, 1, 3, 8, 9, 3, 0, 7, 1, 3, 6, 3, 6, 3, 3, 7, 1, 1, 2, 0, 8, 6, 4, 0, 0, 3, 3, 4, 2, 7, 6, 2, 8, 0, 2, 9, 5, 2, 6, 6, 8, 4, 1, 6, 7, 4, 5, 1, 5, ]
frecuencias_secuencial : [11, 11, 10, 10, 7, 5, 15, 10, 13, 8, ]
frecuencias_paralelo : [11, 11, 10, 10, 7, 5, 14, 10, 12, 7, ]
sumatoria_secuencial: 450
media_secuencial: 4.5
sumatoria_paralelo: 375
media_paralelo: 3.75
mas_grande_secuencial: 26
valor_moda_secuencial: 6
mas_grande_paralelo: 25
valor_moda_paralelo: 6
****************************************************************************************
Valores Reales  :
Tiempo TOTAL     : 0.266075 [ms]
Tiempo encuesta Sec  : 0.00119209 [ms]
Tiempo encuesta Omp  : 0.101805 [ms]

SpeedUp          : (tiempo Secuencia

#4. Tabla de pasos

Paso | Procesador | Funcion | Detalle
------------ | ------------ | ------------- | -------------
1 | CPU | code = ... | definicion de la variable code que contiene el texto del programa en C.
2 | CPU | open() | definicion de la variable usada para la creacion del archivo para escritura con el contenido del texto en la variable code.
3 | CPU | .write(code) | creacion del archivo con el contenido del texto en la variable code.
4 | CPU | .close(code) | cierra el archivo creado.
5 | CPU | !g++ -o encuesta ... | comando shell para la compilación del archivo generado.
6 | CPU | @param | Lectura del tamaño de vector de Colab.
7 | CPU | script_file = ... | Creación de archivo .sh que contendrá el comando para la ejecución del programa junto con el parametro ingresado en el paso 6.
8 | CPU | !chmod 755 ./script_file.sh | comando para otorgar permisos de lectura sobre el archivo creado para que permita su ejecucion.
9 | CPU | !./script_file.sh | ejecución del programa a traves del archivo de script shell creado.

#5. Conclusiones
* ### 5.1 Medidas
##### **Medidas de prestaciones en algoritmos paralelos**
Las tecnicas de HPC buscan reducir los tiempos de ejecución, el tiempo como medida, no alcanza. Dos algoritmos pueden ejecutar en el mismo tiempo, pero uno de ellos usa menos procesadores [4,5].
##### **SpeedUp**
Referencia a la ganacia de velocidad que se consigue con un algoritmo paralelo, al resolver el mismo problema con respecto al algoritmo secuencial.
##### **Eficiencia**
La eficiencia normaliza el valor del SpeedUp, al dividirlo por la cantidad de procesadores que se utilizaron para alcanzar la ganacia en velocidad. Dando la idea de la porción de tiempo que los procesadores se dedican al trabajo útil.
##### **Coste**
El coste de un algoritmo paralelo representa el tiempo realizado por todo el sistema en la resoluciòn del problema.
##### **Función Overhead**
Es la diferencia entre el Coste y el tiempo secuencial. Mientras mayor es la función overhead, peor es el comportamiento del algoritmo paralelo.

* ### 5.2 Breve repaso de los puntos mas relevantes del trabajo.
El algoritmo ejecutado en forma secuencial se lleva tiempo de ejecución sobre cada iteración, entonces el for ocupa la mayor parte del tiempo en la ejecución del programa. Con openMP se paraleliza la ejecución sobre la cantidad de procesadores o nucleos que posee, procesando cada valor del vector en la posición i (índice) en cada uno de los hilos de procesamiento.
Si analizamos los datos obtenidos, por ejemplo, el speeUp, en este caso, luego de varias ejecuciónes el valor ronda los 0.7, lo cual no es muy bueno dado que me esta diciendo que el procesamiento en paralelo, no solo no mejora la ejecución sino que lo hace mas lento en la ejecucion paralelo. La eficiencia tambien esta dando cercana a cero, y esto no es bueno tampoco, debería ser cerana a 1 como ideal. A nivel de recursos, el costo esta arrojando valores identicos para el proceso secuencial y paralelo y no muestra valores significativos de overhead. 

* ### 5.3 Explicación sobre las lecciones aprendidas que deja el ejercicio.
En este caso contamos con dos nucleos, con lo cual el nivel de paralelismo no se llega a apreciar tanto con resultados optimos, ya que en algun punto el procesamiento sera secuencial, en el sentido de que puede llegar a ejecutar solo 2 procesos en paralelo, por esto, por ejemplo en un vector de 100k elementos, podremos ver una mejora al ejecutar secuencialmente 50k veces en lugar de 100k, debido a los dos nucleos. En cambio si contaramos con una cantidad de procesadores mayor, la optimización sería realmente considerable. Esto se puede observar en los valores que arrojan las medidas tomadas descriptas en el punto 5.1.
* ### 5.4 Sugerencias para continuar con el ejercicio (funcionalidad o algoritmo).
En cuanto a la funcionalidad, quisiera continuar obteniendo los datos de la matriz resultados desde un archivo, de manera que se puedan cargar datos verdaderos en lugar de generarlos aleatoriamente. Estos datos podrian llegar a ser generados desde otro programa que capture datos reales, con lo cual podría llegar a tener una utilidad real para analizar datos. Ya no tendría sentido hacer la comparación con el algoritmo secuencial y estaría destinado a otro propósito que no sería el estudio de las herramientas utilizadas. 
En cuanto al código hay muchas cosas para mejorar, ya sea con el refactor del codigo en c++ y el uso de omp. En cuanto al algoritmo se podrían agregar mas datos de las encuestas como la mediana, o bien poder utilizar procesamiento en paralelo para ordenar los datos dentro del vector, etc.


#6. Bibliografía

* [1] Como Programar en C C++ y Java 4ta Edición Harvey M. Deitel & Paul J. Deitel.
* [2] Python Básico - SOA UNLaM: [Python Básico](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb)
* [3] Tutorial Point Colab: [Google Colab Tutorial](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/google_colab_tutorial.pdf)
* [4] F. Almeida, D. Gimenéz, A. Vidal - Introducción a la programación paralela - 2008 - Editorial Parafino.
* [5] D. Jiménez-González - Introducción a las arquitecturas paralelas. [pdf](http://so-unlam.com.ar/material-clase/HPC/Arquitecturas_de_computadores_avanzadas_(Modulo_1)